In [ ]:
import pandas as pd
df=pd.read_csv("data/Telco_Cusomer_Churn.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()
df.describe()
df.isnull().sum()

In [ ]:
df["TotalCharges"].unique()[:20]

In [ ]:
df["TotalCharges"].str.strip().eq("").sum()
df[df["TotalCharges"].str.strip().eq("")]
df=df.drop("customerID",axis=1)

In [ ]:
df["TotalCharges"]=pd.to_numeric(df["TotalCharges"],errors="coerce")
df["TotalCharges"]=df["TotalCharges"].fillna(0)
df["TotalCharges"].isnull().sum()



In [ ]:
x=df.drop("Churn",axis=1)
y=df["Churn"]
y=y.map({"No":0,"Yes":1})
df["Churn"].value_counts()

In [ ]:
df.select_dtypes(include="object").columns
x=pd.get_dummies(x,drop_first="True")
x.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,
                                               test_size=0.2,
                                               random_state=42,
                                               stratify=y)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=1000,class_weight="balanced")
model.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import classification_report
y_pred=model.predict(x_test)
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
cm=confusion_matrix(y_test,y_pred)
sns.heatmap(cm,annot=True,fmt='d')
plt.xlabel("predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score
y_prob=model.predict_proba(x_test)[:,1]
fpr,tpr,threeshold=roc_curve(y_test,y_prob)
plt.plot(fpr,tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()
print("AUC:",roc_auc_score(y_test,y_prob))

In [ ]:
importance=pd.Series(model.coef_[0],index=x.columns)
importance.sort_values(ascending=False).head(10)